In [1]:
!pip install matplotlib_inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
import scipy.stats as ss
from scipy import optimize
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import pandas as pd

**Задание 1**

In [ ]:
sample = np.array(list(map(float, input().strip().split())))
n = len(sample)

t = sample.mean() * np.sqrt(n) / sample.std(ddof=1)

print(2 * (1 - ss.t(n-1).cdf(abs(t))))

**Задание 2**


In [3]:
n = 1000000
samples = ss.expon().rvs(size=(n,30), random_state=42)

def t(sample):
  return np.sqrt(30) * (sample.mean() - 1) / sample.std(ddof=1)

(np.array([t(sample) for sample in samples]) > ss.norm().ppf(0.95)).mean()



0.026222

**Задание 3**

In [4]:
n = 1000000
samples = ss.expon().rvs(size=(n,30), random_state=42)

def t(sample):
  return np.sqrt(30) * (sample.mean() - 1) / sample.std(ddof=1)

np.quantile(np.array([t(sample) for sample in samples]), 0.95)


1.4088717357948108

**Задание 4**

In [61]:
with open('casino.txt', 'r') as file:
    data = []
    for row in file.readlines():
        data.append(list(map(int, row.strip().split())))

np.random.seed(7)

def f(x, data):
    p_win, p_go_after_win, p_go_after_lose = x
    p_lose = 1 - p_win
    res = 0
    for d in data:
      d_ = np.array(d)
      ones = d_[:-1].sum()
      zeros = (d_[:-1] == 0).sum()
      res_ = ones * np.log(p_win * (1 - p_go_after_win)) + zeros * np.log(p_lose * (1 - p_go_after_lose))
      if d[-1] == 1:
        res_ += np.log(p_win * p_go_after_win)
      else:
        res_ += np.log(p_lose * p_go_after_lose)
      res += res_
    return -res

bounds=[(0.0001, .999999) for _ in range(3)]
def T(data):
  res = minimize(lambda x: f(x, data), x0=np.array([1/2,1/2,1/2]), bounds=bounds)
  return res.x[2] - res.x[1]

# t = T(data)
t = 0.41613186745756475
N = 10
n = len(data) #1000

sample_k = ss.geom(0.55).rvs(size=(N,n))
cnt = 0
for sample in sample_k:
  my_data = [ss.bernoulli(p=0.5).rvs(size=k) for k in sample]
  if T(my_data) > t:
    cnt += 1

cnt/N

0.0

**Задание 5**

In [59]:
def T1(x, y):
  return np.corrcoef(np.array([x, y]))[0,1]

def T2(x, y):
  return T1(ss.rankdata(x), ss.rankdata(y))

rho_0 = 0
rho_1 = 1/2
n = 30
N = 10000
samples0_x, samples0_y = ss.multivariate_normal(cov=[[1, rho_0], [rho_0, 1]]).rvs(size=(N, n), random_state=42).T
samples1_x, samples1_y = ss.multivariate_normal(cov=[[1, rho_1], [rho_1, 1]]).rvs(size=(N, n), random_state=42).T


t1 = np.array([T1(sample0_x, sample0_y) for sample0_x, sample0_y in zip(samples0_x.T, samples0_y.T)])
t2 = np.array([T2(sample0_x, sample0_y) for sample0_x, sample0_y in zip(samples0_x.T, samples0_y.T)])

c1 = np.quantile(t1, 0.95)
c2 = np.quantile(t2, 0.95)

(np.array([T1(sample1_x, sample1_y) for sample1_x, sample1_y in zip(samples1_x.T, samples1_y.T)]) > c1).mean(), (np.array([T2(sample1_x, sample1_y) for sample1_x, sample1_y in zip(samples1_x.T, samples1_y.T)]) > c2).mean()

(0.8961, 0.8511)